In [1]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import numpy as np


In [2]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\MUNIC")

In [3]:
df18=pd.read_csv("MUNIC_educacao_2018.csv", sep=';')
df21=pd.read_csv("MUNIC_educacao_2021.csv", sep=';')

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_12508\3916205321.py:1: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df18=pd.read_csv("MUNIC_educacao_2018.csv", sep=';')


In [4]:
df18 = df18[['Cod Municipio', 'genero', 'idade', 'raca_cor',
        'escolaridade']]
df21 = df21[['CodMun', 'genero', 'idade', 'raca_cor',
        'escolaridade']]
        

In [5]:
df18['ano']=2018
df21['ano']=2021

In [6]:
df18 = df18.rename(columns={'Cod Municipio':'id_municipio','CodMun':'id_municipio', 'escolaridade':'grau_instrucao'})
df21 = df21.rename(columns={'Cod Municipio':'id_municipio','CodMun':'id_municipio', 'escolaridade':'grau_instrucao'})

In [7]:
df = pd.concat([df18,df21])

In [8]:
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade'] =pd.to_numeric(df['idade'])

df['raca_cor']=np.where(df['raca_cor']=='Não informou','Sem dados',df['raca_cor'])


In [9]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [10]:
## criando faixa etária
df1= df

In [11]:
### Dataframe do mapa

df1 = df1[['ano','id_municipio', 'genero', 'faixa_etaria', 'raca_cor', 'grau_instrucao']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11140 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   ano             11140 non-null  int64   
 1   id_municipio    11140 non-null  int64   
 2   genero          11140 non-null  object  
 3   faixa_etaria    11101 non-null  category
 4   raca_cor        11140 non-null  object  
 5   grau_instrucao  11140 non-null  object  
dtypes: category(1), int64(2), object(3)
memory usage: 533.3+ KB


In [12]:
df1=df1.rename(columns={'raca_cor':'cor_raca'}) 

In [13]:
df1

,ano,id_municipio,genero,faixa_etaria,cor_raca,grau_instrucao
0,2018,1100015,Masculino,Entre 50-64,Parda,Especialização
1,2018,1100023,Feminino,Entre 30-49,Parda,Especialização
2,2018,1100031,Feminino,Entre 50-64,Parda,Especialização
3,2018,1100049,Feminino,Entre 50-64,Branca,Especialização
4,2018,1100056,Feminino,Entre 50-64,Branca,Especialização
...,...,...,...,...,...,...
5565,2021,5222005,Masculino,Entre 30-49,Parda,Ensino superior completo
5566,2021,5222054,Feminino,Entre 30-49,Branca,Especialização
5567,2021,5222203,Masculino,Entre 30-49,Parda,Especialização
5568,2021,5222302,Feminino,Entre 30-49,Parda,Especialização


In [14]:
uf = pd.read_excel('Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


In [15]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [16]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [17]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [18]:
df

,id_municipio,genero,idade,raca_cor,grau_instrucao,ano,faixa_etaria,CodMun,UF,Cod UF,Mun
0,1100015,Masculino,61.0,Parda,Especialização,2018,Entre 50-64,1100015,RO,11,Alta Floresta DOeste
1,1100015,Masculino,60.0,Parda,Especialização,2021,Entre 50-64,1100015,RO,11,Alta Floresta DOeste
2,1100023,Feminino,45.0,Parda,Especialização,2018,Entre 30-49,1100023,RO,11,Ariquemes
3,1100023,Feminino,56.0,Parda,Especialização,2021,Entre 50-64,1100023,RO,11,Ariquemes
4,1100031,Feminino,59.0,Parda,Especialização,2018,Entre 50-64,1100031,RO,11,Cabixi
...,...,...,...,...,...,...,...,...,...,...,...
11135,5222203,Masculino,41.0,Parda,Especialização,2021,Entre 30-49,5222203,GO,52,Vila Boa
11136,5222302,Masculino,44.0,Parda,Especialização,2018,Entre 30-49,5222302,GO,52,Vila Propício
11137,5222302,Feminino,36.0,Parda,Especialização,2021,Entre 30-49,5222302,GO,52,Vila Propício
11138,5300108,Masculino,64.0,Branca,Especialização,2018,Entre 50-64,5300108,DF,53,Brasília


In [19]:
df = df.drop(['CodMun'], axis=1)

In [20]:
df1['id_municipio']=df1['id_municipio'].apply(str)

In [30]:
df = df.rename(columns={'UF':'sigla_uf',
                        'Cod UF':'cod_uf',
                        'Mun':'nome_municipio',
                        'raca_cor':'cor_raca'}) #padronizando as colunas

In [22]:
df=df[['ano', 'sigla_uf', 'nome_municipio','id_municipio', 'genero', 'faixa_etaria', 'cor_raca',
       'grau_instrucao']]

In [23]:
# criando dicionário
dict_esco = {'Ensino fundamental (1º Grau) incompleto':'Até Ensino Fundamental',
             'Ensino fundamental (1º Grau) completo':'Até Ensino Fundamental',
            'Ensino médio (2º Grau) incompleto':'Até Ensino Médio',
             'Ensino médio (2º Grau) completo':'Até Ensino Médio',
             'Ensino superior incompleto':'Até Ensino Superior Completo',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}


In [24]:
df = df.replace({'grau_instrucao':dict_esco}) #substituindo valores para padronizar

In [25]:
#padronizando os dados da coluna 'genero'
df['genero']=np.where(df['genero']=='Não informou','Sem dados',df['genero'])

#padronizando os dados da coluna 'escolaridade'
df['grau_instrucao']=np.where(df['grau_instrucao']=='Não informou','Sem dados',df['grau_instrucao'])


In [54]:
df.columns

Index(['ano', 'sigla_uf', 'nome_municipio', 'id_municipio', 'genero',
       'faixa_etaria', 'cor_raca', 'grau_instrucao'],
      dtype='object')

In [59]:
df['grau_instrucao'].unique()

array(['Até Pós Graduação ou Mestrado', 'Até Ensino Superior Completo',
       'Até Ensino Médio', 'Até Doutorado', 'Recusa',
       'Até Ensino Fundamental', 'Sem dados'], dtype=object)

Subindo para o GBQ

In [32]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11140 entries, 0 to 11139
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   ano             11140 non-null  int64   
 1   sigla_uf        11140 non-null  object  
 2   nome_municipio  11140 non-null  object  
 3   id_municipio    11140 non-null  int64   
 4   genero          11140 non-null  object  
 5   faixa_etaria    11101 non-null  category
 6   cor_raca        11140 non-null  object  
 7   grau_instrucao  11140 non-null  object  
dtypes: category(1), int64(2), object(5)
memory usage: 707.3+ KB


In [36]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='sigla uf do municipio'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Código do IBGE do município'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')

 ]


In [37]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_educacao_mapa')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=5cfa8f38-0bc9-4f62-b052-ef7fc0a483bb>